In [1]:
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image
import numpy as np
import pandas as pd
import wget
import smtplib
import os
import shutil

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Getdata.csv')

In [3]:
model = load_model('model_covid19.h5')

In [4]:
 def send_mail(sender_mail,sender_pass,receiver_mail,message): 
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login(sender_mail,sender_pass)

    server.sendmail(sender_mail,receiver_mail,message) 

    smtpserver.quit()

In [5]:
def url_to_jpg(ID,file_path,file_name):
    img_format = '.jpg'
    full_path = '{}{}{}'.format(file_path, file_name, img_format)
    wget.download(f'https://docs.google.com/uc?export=download&id={ID}', out = full_path)

In [6]:
def get_result(img):
    img = image.load_img(img,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    out = model.predict_classes(img)
    return out

In [7]:
data.head()

,Timestamp,Email Address,Upload PA view of chest's x-ray image
0,2020/05/10 4:01:52 PM GMT+5:30,sk@gmail.com,https://drive.google.com/u/0/open?usp=forms_we...
1,2020/05/10 4:02:23 PM GMT+5:30,mm@gmail.com,https://drive.google.com/u/0/open?usp=forms_we...
2,2020/05/10 4:02:49 PM GMT+5:30,jj@gmail.com,https://drive.google.com/u/0/open?usp=forms_we...
3,2020/05/10 4:03:20 PM GMT+5:30,ii@gmail.com,https://drive.google.com/u/0/open?usp=forms_we...
4,2020/05/10 4:03:46 PM GMT+5:30,djkf@gamil.com,https://drive.google.com/u/0/open?usp=forms_we...


In [8]:
dir_path = 'F:\MLops'

In [9]:
size = data['Upload PA view of chest\'s x-ray image'].shape[0]

In [10]:
size

5

In [11]:
if os.path.isdir(dir_path):
    os.mkdir(dir_path+'\images')

In [12]:
file_path = dir_path+'\images/'

In [13]:
file_path

'F:\\MLops\\images/'

In [14]:
for i in range(size):
    ID = data['Upload PA view of chest\'s x-ray image'].iloc[i].split('=')[-1]
    file_name = data['Email Address'].iloc[i]
    url_to_jpg(ID,file_path,file_name)

100% [..............................................................................] 63413 / 63413

In [15]:
load_images = os.listdir(file_path)

In [16]:
load_images

['djkf@gamil.com.jpg',
 'ii@gmail.com.jpg',
 'jj@gmail.com.jpg',
 'mm@gmail.com.jpg',
 'sk@gmail.com.jpg']

In [17]:
res = {1:'NORMAL', 0:'COVID Positive'}

In [18]:
res[0]

'COVID Positive'

In [19]:
for img in load_images:
    out = get_result(file_path+img)[0][0]
    recipient_mail = img.split('.jpg')[0]
    message = res[out]
    print('{}\t{}'.format(recipient_mail,message))
    #send_mail(sender_mail,sender_pass,recipient_mail,message):
    

djkf@gamil.com	NORMAL
ii@gmail.com	NORMAL
jj@gmail.com	NORMAL
mm@gmail.com	COVID Positive
sk@gmail.com	COVID Positive


In [20]:
shutil.rmtree(dir_path+'\images')